In [19]:
import numpy
from numba import njit

In [2]:
particle_dtype = numpy.dtype({'names':['x','y','z','m','phi'], 
                             'formats':[numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double]})

# Exercise 1

Write a function `create_n_random_particles` that takes the arguments `n` (number of particles), `m` (mass of every particle) and a domain within to generate a random number (as in the class above).
It should create an array with `n` elements and `dtype=particle_dtype` and then return that array.

For each particle, the mass should be initialized to the value of `m` and the potential `phi` initialized to zero.

For the `x` component of a given particle `p`, you might do something like

```python
p['x'] = domain * numpy.random.random()
```

In [13]:
n=10
numpy.zeros((n), dtype=particle_dtype)[0]['x']

0.0

In [20]:
@njit
def create_n_random_particles(n, m, domain=1):
    '''
    Creates `n` particles with mass `m` with random coordinates
    between 0 and `domain`
    '''
    parts = numpy.zeros((n), dtype=particle_dtype)
    for p in parts:
        p['x'] = numpy.random.random()*domain
        p['y'] = numpy.random.random()*domain
        p['z'] = numpy.random.random()*domain
        p['m'] = m
        p['phi'] = 0
    

    return parts

Test it out!

In [17]:
%timeit parts = create_n_random_particles(1000, .001, 1)

4.6 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [104]:
%timeit parts = create_n_random_particles(1000, .001, 1)

24.3 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Exercise 2

Write a JITted function `distance` to calculate the distance between two particles of dtype `particle_dtype`

Here's the `distance` method from the `Particle` class as a reference:

```python
def distance(self, other):
        return ((self.x - other.x)**2 + 
                (self.y - other.y)**2 + 
                (self.z - other.z)**2)**.5
```

In [32]:
@njit
def distance(part1, part2):
    '''calculate the distance between two particles'''
    
    d2 = (part1['x'] - part2['x'])**2 + (part1['y'] - part2['y'])**2 + (part1['z'] - part2['z'])**2
    d = d2**0.5
    return(d2)

Try it out!

In [33]:
%timeit distance(parts[0], parts[1])

895 ns ± 35.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [31]:
%timeit distance(parts[0], parts[1])

4.56 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The speed up was pretty minimal here, only a factor of 5

# Exercise 3

Modify the original `direct_sum` function (copied below for reference) to instead work a NumPy array of particles.  Loop over each element in the array and calculate its total potential.

```python
def direct_sum(particles):
    """
    Calculate the potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r
```

In [123]:
particles = parts[:5]

In [127]:
@njit
def direct_sum(particles):
    for i, target in enumerate(particles):
        ind = set(range(particles.shape[0]))
        ind.discard(i)
        ind = list(ind)
        # failed! TypingError: Failed at nopython (nopython frontend)
        for source in particles[ind]:
            r = distance(target,source)
            target['phi'] += source['m'] / r
    return(particles)
    

In [128]:
direct_sum(particles)

TypingError: Failed at nopython (nopython frontend)
cannot unify set(int64) and list(int64) for 'ind', defined at <ipython-input-127-58b1938a3f79> (4)
File "<ipython-input-127-58b1938a3f79>", line 6
[1] During: typing of assignment at <ipython-input-127-58b1938a3f79> (6)

The function failed above because we were trying to get all the other particles in source without using another loop. 
There are ways of defining source to avoid another loop, but `jit` cannot optmize the ones I have tried.
We can naively use `enumberate` twice and do not throw any nopython error. Let's go ahead with this and see what the speed up is

In [129]:
@njit
def direct_sum(particles):

    for i, target in enumerate(particles):
        for j, source in enumerate(particles):
            if i != j:
                r = distance(target,source)
                target['phi'] += source['m'] / r            
            
    return(particles)
    

In [131]:
%timeit direct_sum(parts)

4.23 ms ± 30.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [132]:
def direct_sum(particles):

    for i, target in enumerate(particles):
        for j, source in enumerate(particles):
            if i != j:
                r = distance(target,source)
                target['phi'] += source['m'] / r            
            
    return(particles)
    

In [133]:
%timeit direct_sum(parts)

2.43 s ± 14.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


That's a 574x speedup. It is exagerated because the non jitted function is so poorly optimized